# Interactive Graph Visualization with Dijkstra's Algorithm

This notebook creates an interactive graph visualization with:
- Hover over edges to see: start → end and weight
- Hover over nodes to see: all connections sorted by node value
- **Dijkstra's shortest path tree highlighted in green**

In [1]:
# Install required packages (run once)
# !pip install plotly networkx

In [2]:
import plotly.graph_objects as go
import networkx as nx
from graph import read_file
from djikstra import Dijkstra
import numpy as np
import sys
from io import StringIO

In [3]:
def create_interactive_graph(graph, initial_node=None, max_nodes=50, dijkstra_paths=None):
    """
    Create an interactive graph visualization with detailed hover information
    
    Args:
        graph: Dictionary {node: {neighbor: weight, ...}}
        initial_node: Starting node to highlight
        max_nodes: Maximum number of nodes to display
        dijkstra_paths: Dictionary of shortest paths from Dijkstra's algorithm
    """
    # Create directed graph
    G = nx.DiGraph()
    
    # Limit nodes for visualization
    nodes_to_show = sorted(graph.keys())[:max_nodes]
    
    # Add edges with weights (only positive weights > 0)
    for node in nodes_to_show:
        if node in graph:
            for neighbor, weight in graph[node].items():
                if neighbor in nodes_to_show and weight > 0:
                    G.add_edge(node, neighbor, weight=weight)
    
    # Create layout
    pos = nx.spring_layout(G, k=3, iterations=50, seed=42)
    
    # Extract Dijkstra edges (shortest path tree)
    dijkstra_edges = set()
    if dijkstra_paths:
        for node, path in dijkstra_paths.items():
            if node in nodes_to_show and len(path) > 1:
                # Add edges from the path
                for i in range(len(path) - 1):
                    if path[i] in nodes_to_show and path[i+1] in nodes_to_show:
                        dijkstra_edges.add((path[i], path[i+1]))
    
    # Create edge traces with hover information
    edge_traces = []
    
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        weight = edge[2]['weight']
        
        # Check if this edge is part of Dijkstra's shortest path tree
        is_dijkstra_edge = (edge[0], edge[1]) in dijkstra_edges
        
        # Set color and width based on whether it's a Dijkstra edge
        edge_color = '#00CC00' if is_dijkstra_edge else '#888'
        edge_width = 3 if is_dijkstra_edge else 1
        
        # Create edge line
        edge_trace = go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=edge_width, color=edge_color),
            hoverinfo='text',
            text=f'Edge: {edge[0]} → {edge[1]}<br>Weight: {weight:.2f}' + 
                 ('<br><b>✓ Shortest Path Edge</b>' if is_dijkstra_edge else ''),
            showlegend=False
        )
        edge_traces.append(edge_trace)
        
        # Add arrow marker at midpoint
        mid_x = (x0 + x1) / 2
        mid_y = (y0 + y1) / 2
        
        # Calculate arrow direction
        dx = x1 - x0
        dy = y1 - y0
        angle = np.arctan2(dy, dx) * 180 / np.pi
        
        arrow_trace = go.Scatter(
            x=[mid_x],
            y=[mid_y],
            mode='markers',
            marker=dict(
                symbol='arrow',
                size=15 if is_dijkstra_edge else 10,
                color=edge_color,
                angle=angle,
                angleref='previous'
            ),
            hoverinfo='text',
            text=f'Edge: {edge[0]} → {edge[1]}<br>Weight: {weight:.2f}' + 
                 ('<br><b>✓ Shortest Path Edge</b>' if is_dijkstra_edge else ''),
            showlegend=False
        )
        edge_traces.append(arrow_trace)
    
    # Create node hover text with all connections sorted by node value
    node_hover_text = []
    node_colors = []
    
    for node in G.nodes():
        # Get all outgoing edges from this node
        connections = []
        if node in graph:
            # Sort connections by neighbor node value
            sorted_neighbors = sorted(graph[node].items(), key=lambda x: x[0])
            for neighbor, weight in sorted_neighbors:
                if neighbor in nodes_to_show:
                    connections.append(f'  → Node {neighbor}: {weight:.2f}')
        
        # Create hover text
        hover_text = f'<b>Node {node}</b><br>'
        
        # Add Dijkstra path information if available
        if dijkstra_paths and node in dijkstra_paths:
            path = dijkstra_paths[node]
            path_str = ' → '.join(str(n) for n in path)
            hover_text += f'<b>Shortest Path from {initial_node}:</b><br>{path_str}<br>'
        
        hover_text += f'<b>Outgoing Connections:</b> {len(connections)}<br>'
        if connections:
            hover_text += '<br>'.join(connections[:20])  # Limit to first 20
            if len(connections) > 20:
                hover_text += f'<br>... and {len(connections) - 20} more'
        else:
            hover_text += 'No outgoing connections'
        
        node_hover_text.append(hover_text)
        
        # Color initial node differently
        if node == initial_node:
            node_colors.append('red')
        else:
            node_colors.append('lightblue')
    
    # Create node trace
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=[str(node) for node in G.nodes()],
        textposition='top center',
        textfont=dict(size=10, color='black'),
        marker=dict(
            size=20,
            color=node_colors,
            line=dict(width=2, color='darkblue')
        ),
        hoverinfo='text',
        hovertext=node_hover_text,
        showlegend=False
    )
    
    # Create figure
    fig = go.Figure(
        data=edge_traces + [node_trace],
        layout=go.Layout(
            title=dict(
                text=f'Interactive Graph Visualization with Dijkstra\'s Algorithm<br>'
                     f'<sub>Showing {len(nodes_to_show)} of {len(graph)} nodes | '
                     f'Initial Node: {initial_node} | '
                     f'<span style="color:green">Green = Shortest Path</span></sub>',
                x=0.5,
                xanchor='center'
            ),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=80),
            xaxis=dict(
                showgrid=False,
                zeroline=False,
                showticklabels=False
            ),
            yaxis=dict(
                showgrid=False,
                zeroline=False,
                showticklabels=False
            ),
            plot_bgcolor='white',
            width=1200,
            height=800
        )
    )
    
    return fig

In [4]:
# Load the graph
graph, initial_node = read_file('casos_teste_v3/caso_teste_pequeno_1.txt')

print(f"Graph loaded: {len(graph)} nodes")
print(f"Initial node: {initial_node}")

Graph loaded: 100 nodes
Initial node: 69


In [5]:
# Run Dijkstra's algorithm (suppress print output)
print("Running Dijkstra's algorithm...")

# Capture stdout to suppress the algorithm's print statements
old_stdout = sys.stdout
sys.stdout = StringIO()

dijkstra = Dijkstra(graph, initial_node)
distances, visited = dijkstra.dijkstra()

# Restore stdout
sys.stdout = old_stdout

print(f"Dijkstra completed! Found shortest paths to {len(visited)} nodes")
print(f"\nExample paths:")
for i, (node, path) in enumerate(list(visited.items())[:5]):
    path_str = ' → '.join(str(n) for n in path)
    print(f"  Node {node}: {path_str} (distance: {distances[node]:.2f})")

Running Dijkstra's algorithm...


ValueError: too many values to unpack (expected 2)

In [ ]:
"""
Interactive Graph Visualization with Dijkstra's Algorithm
Shows bidirectional arrows (→ ←) when edges exist in both directions
"""

import plotly.graph_objects as go
import networkx as nx
from graph import read_file
from djikstra import Dijkstra
import numpy as np
import sys
from io import StringIO


def create_interactive_graph(graph, initial_node=None, max_nodes=50, dijkstra_paths=None):
    """
    Create an interactive graph visualization with detailed hover information
    
    Args:
        graph: Dictionary {node: {neighbor: weight, ...}}
        initial_node: Starting node to highlight
        max_nodes: Maximum number of nodes to display
        dijkstra_paths: Dictionary of shortest paths from Dijkstra's algorithm
    """
    # Create directed graph
    G = nx.DiGraph()
    
    # Limit nodes for visualization
    nodes_to_show = sorted(graph.keys())[:max_nodes]
    
    # Add edges with weights (only positive weights > 0)
    for node in nodes_to_show:
        if node in graph:
            for neighbor, weight in graph[node].items():
                if neighbor in nodes_to_show and weight > 0:
                    G.add_edge(node, neighbor, weight=weight)
    
    # Create layout
    pos = nx.spring_layout(G, k=3, iterations=50, seed=42)
    
    # Extract Dijkstra edges (shortest path tree)
    dijkstra_edges = set()
    if dijkstra_paths:
        for node, path in dijkstra_paths.items():
            if node in nodes_to_show and len(path) > 1:
                # Add edges from the path
                for i in range(len(path) - 1):
                    if path[i] in nodes_to_show and path[i+1] in nodes_to_show:
                        dijkstra_edges.add((path[i], path[i+1]))
    
    # Create edge traces with hover information
    edge_traces = []
    
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        weight = edge[2]['weight']
        
        # Check if reverse edge exists (bidirectional)
        reverse_exists = G.has_edge(edge[1], edge[0])
        reverse_weight = G[edge[1]][edge[0]]['weight'] if reverse_exists else None
        
        # Check if this edge is part of Dijkstra's shortest path tree
        is_dijkstra_edge = (edge[0], edge[1]) in dijkstra_edges
        
        # Set color and width based on whether it's a Dijkstra edge
        edge_color = '#00CC00' if is_dijkstra_edge else '#888'
        edge_width = 3 if is_dijkstra_edge else 1
        
        # Create hover text with bidirectional indicator
        if reverse_exists:
            hover_text = f'Edge: {edge[0]} ⇄ {edge[1]}<br>Forward ({edge[0]}→{edge[1]}): {weight:.2f}<br>Backward ({edge[1]}→{edge[0]}): {reverse_weight:.2f}'
        else:
            hover_text = f'Edge: {edge[0]} → {edge[1]}<br>Weight: {weight:.2f}'
        
        if is_dijkstra_edge:
            hover_text += '<br><b>✓ Shortest Path Edge</b>'
        
        # Create edge line
        edge_trace = go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=edge_width, color=edge_color),
            hoverinfo='text',
            text=hover_text,
            showlegend=False
        )
        edge_traces.append(edge_trace)
        
        # Add multiple arrow markers along the edge for better visibility
        # Place arrows at 30%, 50%, and 70% along the edge
        for position in [0.3, 0.5, 0.7]:
            arrow_x = x0 + (x1 - x0) * position
            arrow_y = y0 + (y1 - y0) * position
            
            # Calculate arrow direction
            dx = x1 - x0
            dy = y1 - y0
            angle = np.arctan2(dy, dx) * 180 / np.pi
            
            arrow_trace = go.Scatter(
                x=[arrow_x],
                y=[arrow_y],
                mode='markers',
                marker=dict(
                    symbol='arrow',
                    size=20 if is_dijkstra_edge else 15,
                    color=edge_color,
                    angle=angle,
                    angleref='previous',
                    line=dict(width=1, color='white')
                ),
                hoverinfo='text',
                text=hover_text,
                showlegend=False
            )
            edge_traces.append(arrow_trace)
    
    # Create node hover text with all connections sorted by node value
    node_hover_text = []
    node_colors = []
    
    for node in G.nodes():
        # Get all outgoing edges from this node
        connections = []
        if node in graph:
            # Sort connections by neighbor node value
            sorted_neighbors = sorted(graph[node].items(), key=lambda x: x[0])
            for neighbor, weight in sorted_neighbors:
                if neighbor in nodes_to_show and weight > 0:
                    # Check if bidirectional
                    if neighbor in graph and node in graph[neighbor] and graph[neighbor][node] > 0:
                        connections.append(f'  ⇄ Node {neighbor}: {weight:.2f} (bidirectional)')
                    else:
                        connections.append(f'  → Node {neighbor}: {weight:.2f}')
        
        # Create hover text
        hover_text = f'<b>Node {node}</b><br>'
        
        # Add Dijkstra path information if available
        if dijkstra_paths and node in dijkstra_paths:
            path = dijkstra_paths[node]
            path_str = ' → '.join(str(n) for n in path)
            hover_text += f'<b>Shortest Path from {initial_node}:</b><br>{path_str}<br>'
        
        hover_text += f'<b>Outgoing Connections:</b> {len(connections)}<br>'
        if connections:
            hover_text += '<br>'.join(connections[:20])  # Limit to first 20
            if len(connections) > 20:
                hover_text += f'<br>... and {len(connections) - 20} more'
        else:
            hover_text += 'No outgoing connections'
        
        node_hover_text.append(hover_text)
        
        # Color initial node differently
        if node == initial_node:
            node_colors.append('red')
        else:
            node_colors.append('lightblue')
    
    # Create node trace
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=[str(node) for node in G.nodes()],
        textposition='top center',
        textfont=dict(size=10, color='black'),
        marker=dict(
            size=20,
            color=node_colors,
            line=dict(width=2, color='darkblue')
        ),
        hoverinfo='text',
        hovertext=node_hover_text,
        showlegend=False
    )
    
    # Create figure
    fig = go.Figure(
        data=edge_traces + [node_trace],
        layout=go.Layout(
            title=dict(
                text=f'Interactive Graph Visualization with Dijkstra\'s Algorithm<br>'
                     f'<sub>Showing {len(nodes_to_show)} of {len(graph)} nodes | '
                     f'Initial Node: {initial_node} | '
                     f'<span style="color:green">Green = Shortest Path</span> | '
                     f'⇄ = Bidirectional</sub>',
                x=0.5,
                xanchor='center'
            ),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=80),
            xaxis=dict(
                showgrid=False,
                zeroline=False,
                showticklabels=False
            ),
            yaxis=dict(
                showgrid=False,
                zeroline=False,
                showticklabels=False
            ),
            plot_bgcolor='white',
            width=1200,
            height=800
        )
    )
    
    return fig


if __name__ == "__main__":
    # Load the graph
    graph, initial_node = read_file('casos_teste_v3/caso_teste_pequeno_1.txt')
    
    print(f"Graph loaded: {len(graph)} nodes")
    print(f"Initial node: {initial_node}")
    
    # Run Dijkstra's algorithm (suppress print output)
    print("Running Dijkstra's algorithm...")
    
    # Capture stdout to suppress the algorithm's print statements
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    
    dijkstra = Dijkstra(graph, initial_node)
    distances, visited = dijkstra.dijkstra()
    
    # Restore stdout
    sys.stdout = old_stdout
    
    print(f"Dijkstra completed! Found shortest paths to {len(visited)} nodes")
    print(f"\nExample paths:")
    for i, (node, path) in enumerate(list(visited.items())[:5]):
        path_str = ' → '.join(str(n) for n in path)
        print(f"  Node {node}: {path_str} (distance: {distances[node]:.2f})")
    
    # Create and display the interactive graph with Dijkstra paths
    # Adjust max_nodes to show more or fewer nodes (default: 50)
    fig = create_interactive_graph(graph, initial_node, max_nodes=30, dijkstra_paths=visited)
    fig.show()
    
    # Optionally save to HTML file
    fig.write_html('interactive_graph_dijkstra.html')
    print("\n✓ Graph saved to 'interactive_graph_dijkstra.html'")
    print("\nBidirectional edges are marked with ⇄ symbol")
    print("Hover over edges to see: Forward and Backward weights")
    print("Hover over nodes to see: ⇄ for bidirectional connections")


Graph loaded: 100 nodes
Initial node: 69
Running Dijkstra's algorithm...
Dijkstra completed! Found shortest paths to 100 nodes

Example paths:
  Node 69: 69 (distance: inf)
  Node 30: 69 → 30 (distance: 294.40)
  Node 5: 69 → 30 → 5 (distance: 316.80)
  Node 14: 69 → 30 → 14 (distance: 339.20)
  Node 25: 69 → 30 → 5 → 25 (distance: 352.00)



✓ Graph saved to 'interactive_graph_dijkstra.html'

Bidirectional edges are marked with ⇄ symbol
Hover over edges to see: Forward and Backward weights
Hover over nodes to see: ⇄ for bidirectional connections


In [ ]:
"""
Interactive Graph Visualization - SHORTEST PATH ONLY
Shows ONLY the edges that are part of Dijkstra's shortest path tree
"""

import plotly.graph_objects as go
import networkx as nx
from graph import read_file
from djikstra import Dijkstra
import numpy as np
import sys
from io import StringIO


def create_shortest_path_graph(graph, initial_node=None, max_nodes=50, dijkstra_paths=None):
    """
    Create an interactive graph visualization showing ONLY shortest path edges
    
    Args:
        graph: Dictionary {node: {neighbor: weight, ...}}
        initial_node: Starting node to highlight
        max_nodes: Maximum number of nodes to display
        dijkstra_paths: Dictionary of shortest paths from Dijkstra's algorithm
    """
    # Create directed graph
    G = nx.DiGraph()
    
    # Limit nodes for visualization
    nodes_to_show = sorted(graph.keys())[:max_nodes]
    
    # Extract Dijkstra edges (shortest path tree) FIRST
    dijkstra_edges = set()
    if dijkstra_paths:
        for node, path in dijkstra_paths.items():
            if node in nodes_to_show and len(path) > 1:
                # Add edges from the path
                for i in range(len(path) - 1):
                    if path[i] in nodes_to_show and path[i+1] in nodes_to_show:
                        dijkstra_edges.add((path[i], path[i+1]))
    
    # Add ONLY shortest path edges to the graph
    for node in nodes_to_show:
        if node in graph:
            for neighbor, weight in graph[node].items():
                if neighbor in nodes_to_show and weight > 0:
                    # Only add if it's part of the shortest path tree
                    if (node, neighbor) in dijkstra_edges:
                        G.add_edge(node, neighbor, weight=weight)
    
    # Create layout
    pos = nx.spring_layout(G, k=3, iterations=50, seed=42)
    
    # Create edge traces with hover information
    edge_traces = []
    
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        weight = edge[2]['weight']
        
        # Check if reverse edge exists (bidirectional)
        reverse_exists = G.has_edge(edge[1], edge[0])
        reverse_weight = G[edge[1]][edge[0]]['weight'] if reverse_exists else None
        
        # All edges are shortest path edges in this visualization
        edge_color = '#00CC00'
        edge_width = 4
        
        # Create hover text with bidirectional indicator
        if reverse_exists:
            hover_text = f'<b>Shortest Path Edge</b><br>Edge: {edge[0]} ⇄ {edge[1]}<br>Forward ({edge[0]}→{edge[1]}): {weight:.2f}<br>Backward ({edge[1]}→{edge[0]}): {reverse_weight:.2f}'
        else:
            hover_text = f'<b>Shortest Path Edge</b><br>Edge: {edge[0]} → {edge[1]}<br>Weight: {weight:.2f}'
        
        # Create edge line
        edge_trace = go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=edge_width, color=edge_color),
            hoverinfo='text',
            text=hover_text,
            showlegend=False
        )
        edge_traces.append(edge_trace)
        
        # Add multiple arrow markers along the edge for better visibility
        # Place arrows at 30%, 50%, and 70% along the edge
        for position in [0.3, 0.5, 0.7]:
            arrow_x = x0 + (x1 - x0) * position
            arrow_y = y0 + (y1 - y0) * position
            
            # Calculate arrow direction
            dx = x1 - x0
            dy = y1 - y0
            angle = np.arctan2(dy, dx) * 180 / np.pi
            
            arrow_trace = go.Scatter(
                x=[arrow_x],
                y=[arrow_y],
                mode='markers',
                marker=dict(
                    symbol='arrow',
                    size=20,
                    color=edge_color,
                    angle=angle,
                    angleref='previous',
                    line=dict(width=1, color='white')
                ),
                hoverinfo='text',
                text=hover_text,
                showlegend=False
            )
            edge_traces.append(arrow_trace)
    
    # Create node hover text with shortest path information
    node_hover_text = []
    node_colors = []
    
    for node in G.nodes():
        # Create hover text
        hover_text = f'<b>Node {node}</b><br>'
        
        # Add Dijkstra path information if available
        if dijkstra_paths and node in dijkstra_paths:
            path = dijkstra_paths[node]
            path_str = ' → '.join(str(n) for n in path)
            hover_text += f'<b>Shortest Path from {initial_node}:</b><br>{path_str}<br>'
        
        # Get shortest path edges from this node
        sp_connections = []
        for neighbor in G.neighbors(node):
            weight = G[node][neighbor]['weight']
            # Check if bidirectional
            if G.has_edge(neighbor, node):
                sp_connections.append(f'  ⇄ Node {neighbor}: {weight:.2f} (bidirectional)')
            else:
                sp_connections.append(f'  → Node {neighbor}: {weight:.2f}')
        
        hover_text += f'<b>Shortest Path Connections:</b> {len(sp_connections)}<br>'
        if sp_connections:
            hover_text += '<br>'.join(sp_connections)
        else:
            hover_text += 'No outgoing connections in shortest path tree'
        
        node_hover_text.append(hover_text)
        
        # Color initial node differently
        if node == initial_node:
            node_colors.append('red')
        else:
            node_colors.append('lightgreen')
    
    # Create node trace
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=[str(node) for node in G.nodes()],
        textposition='top center',
        textfont=dict(size=12, color='black', family='Arial Black'),
        marker=dict(
            size=25,
            color=node_colors,
            line=dict(width=3, color='darkgreen')
        ),
        hoverinfo='text',
        hovertext=node_hover_text,
        showlegend=False
    )
    
    # Create figure
    fig = go.Figure(
        data=edge_traces + [node_trace],
        layout=go.Layout(
            title=dict(
                text=f'<b>Shortest Path Tree Visualization</b><br>'
                     f'<sub>Showing ONLY shortest path edges | '
                     f'Nodes: {len(G.nodes())} | '
                     f'Initial Node: {initial_node} | '
                     f'<span style="color:green">All edges are shortest paths</span></sub>',
                x=0.5,
                xanchor='center',
                font=dict(size=18)
            ),
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=5, r=5, t=100),
            xaxis=dict(
                showgrid=False,
                zeroline=False,
                showticklabels=False
            ),
            yaxis=dict(
                showgrid=False,
                zeroline=False,
                showticklabels=False
            ),
            plot_bgcolor='#f0f8f0',
            width=1200,
            height=800
        )
    )
    
    return fig


if __name__ == "__main__":
    # Load the graph
    graph, initial_node = read_file('casos_teste_v3/caso_teste_medio_1.txt')
    
    print(f"Graph loaded: {len(graph)} nodes")
    print(f"Initial node: {initial_node}")
    
    # Run Dijkstra's algorithm (suppress print output)
    print("Running Dijkstra's algorithm...")
    
    # Capture stdout to suppress the algorithm's print statements
    old_stdout = sys.stdout
    sys.stdout = StringIO()
    
    dijkstra = Dijkstra(graph, initial_node)
    distances, visited = dijkstra.dijkstra()
    
    # Restore stdout
    sys.stdout = old_stdout
    
    print(f"Dijkstra completed! Found shortest paths to {len(visited)} nodes")
    print(f"\nShortest paths:")
    for node, path in sorted(visited.items()):
        path_str = ' → '.join(str(n) for n in path)
        print(f"  Node {node}: {path_str} (distance: {distances[node]:.2f})")
    
    # Create and display the shortest path only graph
    fig = create_shortest_path_graph(graph, initial_node, max_nodes=10000, dijkstra_paths=visited)
    fig.show()
    
    # Save to HTML file
    fig.write_html('shortest_path_only.html')
    print("\n✓ Shortest path tree saved to 'shortest_path_only.html'")
    print("\n📊 This visualization shows ONLY the edges in the shortest path tree")
    print("   - All edges shown are part of the optimal paths from the initial node")
    print("   - Green edges with prominent arrows show direction")
    print("   - Red node = starting point, Light green nodes = reachable nodes")


Graph loaded: 5000 nodes
Initial node: 4044
Running Dijkstra's algorithm...
Dijkstra completed! Found shortest paths to 5000 nodes

Shortest paths:
  Node 0: 4044 → 512 → 4650 → 3954 → 1041 → 0 (distance: 25.60)
  Node 1: 4044 → 512 → 4453 → 3713 → 2302 → 4216 → 4513 → 1 (distance: 26.40)
  Node 2: 4044 → 512 → 4453 → 2566 → 2 (distance: 13.60)
  Node 3: 4044 → 512 → 4453 → 2430 → 3 (distance: 23.20)
  Node 4: 4044 → 2311 → 3032 → 3591 → 4 (distance: 20.80)
  Node 5: 4044 → 512 → 4453 → 2566 → 1164 → 3845 → 4673 → 897 → 5 (distance: 26.40)
  Node 6: 4044 → 512 → 4453 → 2566 → 2549 → 1597 → 4521 → 6 (distance: 29.60)
  Node 7: 4044 → 512 → 4453 → 3713 → 1203 → 774 → 720 → 7 (distance: 26.40)
  Node 8: 4044 → 512 → 4453 → 3713 → 2302 → 4216 → 185 → 8 (distance: 24.80)
  Node 9: 4044 → 512 → 4453 → 2566 → 1164 → 3845 → 4673 → 4539 → 835 → 9 (distance: 36.00)
  Node 10: 4044 → 512 → 4453 → 476 → 1736 → 3990 → 10 (distance: 36.00)
  Node 11: 4044 → 512 → 4453 → 3713 → 2608 → 800 → 11 (dista